In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py -O minsearch.py

--2025-03-15 18:17:36--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-03-15 18:17:36 (58.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)


In [5]:
documents = []

for course in docs_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question","text","section"], keyyword_fields=["course"]
    )

In [6]:
q = 'the course has alrady started, can I sitll enroll?'

In [7]:
index = minsearch.Index( text_fields=["question","text","section"], keyword_fields=["course"] )

In [8]:
index.fit(documents)

In [9]:
boost = {'question':2.0, 'text':1.3, 'section':0.5}

results = index.search(
query = q,
boost_dict=boost,
num_results=5
)
    

In [10]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoom

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI()

In [13]:

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}])

In [14]:
response

ChatCompletion(id='chatcmpl-BBQMrztuLo6lJGlQexTtkvLBoRYWp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='If the course has already started, whether you can still enroll depends on several factors:\n\n1. **Institution Policies:** Check the policies of the institution offering the course. Some institutions allow late enrollment within a certain time frame, while others have strict deadlines.\n\n2. **Course Type:** For online or self-paced courses, late enrollment might be possible since you can catch up at your own pace. For in-person or synchronous online courses, catching up might be more challenging.\n\n3. **Instructor Approval:** Sometimes, gaining permission directly from the instructor can allow you to enroll late, especially if you have a compelling reason.\n\n4. **Impact on Learning:** Consider how much content you have missed and whether you can realistically catch up without negatively impacting your learning or performanc

In [15]:
prompt_template = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when awsering. 
Do not ignore this condition. CONTEXT need to be provided.
If CONTEXT does not contain the anser, output 42.

QUESTION :{question}

CONTEXT:
{context}

""".strip()

In [16]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [17]:
prompt = prompt_template.format(question=q, context=context).strip()

In [18]:
print(prompt)

You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when awsering. 
Do not ignore this condition. CONTEXT need to be provided.
If CONTEXT does not contain the anser, output 42.

QUESTION :the course has alrady started, can I sitll enroll?

CONTEXT:
section: General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the cour

In [19]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}])

In [20]:
response.choices[0].message.content

'Yes, you can still enroll in the course after it has started. However, you won’t be able to submit some of the homeworks. To get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline.'

In [21]:
prompt = prompt_template.format(question=q, context="").strip()

In [22]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}])

In [23]:
response.choices[0].message.content

'42'

In [24]:
prompt = prompt_template.format(question=q, context="Im amazing and this is what you have, some other stufff, I don't know what to do.").strip()

In [25]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}])


In [26]:
response.choices[0].message.content

'42'

In [77]:
def search(query):
    boost = {'question':2.0, 'text':1.3, 'section':0.5}

    results = index.search(
    query = query,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=10
    )
    return results

In [78]:
def build_prompt(query, search_result):
    prompt_template = """
    You are a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT when awsering. 
    Do not ignore this condition. CONTEXT need to be provided.
    
    QUESTION :{question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_result:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
    

In [79]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}])
    return response.choices[0].message.content

In [80]:
query = 'How do I run Kafka?'
search_result = search(query)
prompt = build_prompt(query, search_result)
llm(prompt)

'To run Kafka, you should first ensure that all necessary docker containers are up and running. Use the following steps:\n\n1. **Check Docker containers:**\n   - Use the command `docker ps` to confirm that the Kafka broker docker container is active.\n   \n2. **Start Docker instances:**\n   - If the Kafka broker is not running, navigate to the folder containing your docker compose YAML file and execute `docker compose up -d`. This starts all the required instances.\n\nMake sure you also have your environment set up for running any related Kafka scripts, such as ensuring you are in the appropriate virtual environment or Docker environment if needed.'

In [81]:
def rag(query):
    search_result = search(query)
    prompt = build_prompt(query, search_result)
    result = llm(prompt)
    return result

In [82]:
rag('How can I run kafka?')

'To run Kafka, depending on your system and the context, you might need to:\n\n1. **Create and Use a Virtual Environment (for Python):**\n   - Create a virtual environment:\n     ```bash\n     python -m venv env\n     ```\n   - Activate the virtual environment:\n     - On MacOS/Linux:\n       ```bash\n       source env/bin/activate\n       ```\n     - On Windows:\n       ```bash\n       env\\Scripts\\activate\n       ```\n   - Install dependencies from `requirements.txt`:\n     ```bash\n     pip install -r ../requirements.txt\n     ```\n\n2. **For Java Applications:**\n   - Run your Java application from the terminal using the designated command format. For example:\n     ```bash\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n3. **Using Docker (for Kafka and Related Tools):**\n   - Ensure your Kafka broker is running. If encountering `kafka.errors.NoBrokersAvailable`, check that your Kafka broker Docker container is op

In [83]:
from elasticsearch import Elasticsearch

In [84]:
elastic_client = Elasticsearch('http://localhost:9200')

In [89]:
index_settings=    {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [90]:
index_name = 'course-questions'

In [91]:
elastic_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [96]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    elastic_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [99]:
query = 'I just discover the course, can I join'
search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [100]:
elastic_client.search(index=index_name, body=search_query)

ObjectApiResponse({'took': 1080, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 398, 'relation': 'eq'}, 'max_score': 59.710346, 'hits': [{'_index': 'course-questions', '_id': 'HtEpm5UB9SmER-lH6snI', '_score': 59.710346, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'INEpm5UB9SmER-lH6snS', '_score': 43.841484, '_source': {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syl

In [103]:
def search_elastic(query): 
    search_query={
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}
    result = elastic_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in result['hits']['hits']:
       result_docs.append(hit['_source'])
    return result_docs                     
        
    

In [104]:
search_elastic(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

In [105]:
def rag_es(query):
    search_result = search_elastic(query)
    prompt = build_prompt(query, search_result)
    result = llm(prompt)
    return result

In [106]:
rag_es('The course is stared, can I still enroll?')

'Yes, you can still enroll in the course even after it has started. You are eligible to submit the homeworks and participate, but be mindful of the deadlines for final projects.'